In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import os

# Detect GPU
device_list = tf.config.list_physical_devices('GPU')
print("🖥️ Device in use:", device_list if device_list else "⚠️ No GPU found — running on CPU.")

# Parameters
IMG_SIZE = 64
BATCH_SIZE = 32
LATENT_DIM = 100
EPOCHS = 5
DATA_PATH = "training_set"  # Your dataset folder with 'cats/' and 'dogs/'

# Load and preprocess dataset
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    DATA_PATH,
    label_mode=None,
    image_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE
).map(lambda x: (tf.cast(x, tf.float32) / 127.5) - 1).shuffle(1000)

# Generator
def build_generator():
    model = tf.keras.Sequential([
        layers.Dense(8*8*256, input_shape=(LATENT_DIM,), use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Reshape((8, 8, 256)),
        layers.Conv2DTranspose(128, 5, strides=2, padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Conv2DTranspose(64, 5, strides=2, padding='same', use_bias=False),
        layers.BatchNormalization(),
        layers.LeakyReLU(),
        layers.Conv2DTranspose(3, 5, strides=2, padding='same', use_bias=False, activation='tanh')
    ])
    return model

# Discriminator
def build_discriminator():
    model = tf.keras.Sequential([
        layers.Conv2D(64, 5, strides=2, padding='same', input_shape=[IMG_SIZE, IMG_SIZE, 3]),
        layers.LeakyReLU(),
        layers.Dropout(0.3),
        layers.Conv2D(128, 5, strides=2, padding='same'),
        layers.LeakyReLU(),
        layers.Dropout(0.3),
        layers.Flatten(),
        layers.Dense(1)
    ])
    return model

# Instantiate models
generator = build_generator()
discriminator = build_discriminator()
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)
gen_opt = tf.keras.optimizers.Adam(1e-4)
disc_opt = tf.keras.optimizers.Adam(1e-4)

# Training step
@tf.function
def train_step(images):
    noise = tf.random.normal([BATCH_SIZE, LATENT_DIM])
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated = generator(noise, training=True)
        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated, training=True)
        gen_loss = cross_entropy(tf.ones_like(fake_output), fake_output)
        disc_loss = cross_entropy(tf.ones_like(real_output), real_output) + \
                    cross_entropy(tf.zeros_like(fake_output), fake_output)
    gen_grads = gen_tape.gradient(gen_loss, generator.trainable_variables)
    disc_grads = disc_tape.gradient(disc_loss, discriminator.trainable_variables)
    gen_opt.apply_gradients(zip(gen_grads, generator.trainable_variables))
    disc_opt.apply_gradients(zip(disc_grads, discriminator.trainable_variables))
    return gen_loss, disc_loss

# Save generated images
def save_and_show_images(model, epoch, n=4):
    noise = tf.random.normal([n, LATENT_DIM])
    images = model(noise, training=False)
    images = (images + 1) / 2.0

    fig, axs = plt.subplots(2, 2, figsize=(6,6))
    for i in range(n):
        axs[i//2, i%2].imshow(images[i].numpy())
        axs[i//2, i%2].axis('off')
    plt.tight_layout()
    filename = f"generated_epoch_{epoch}.png"
    plt.savefig(filename)
    print(f"🖼️ Saved & showing: {filename}")
    plt.show()  # This displays the image inline


# Training loop
def train(dataset, epochs):
    for epoch in range(epochs):
        for batch in dataset.take(50):  # Short batches for CPU
            gen_loss, disc_loss = train_step(batch)
        print(f"✅ Epoch {epoch}, Gen Loss: {gen_loss:.4f}, Disc Loss: {disc_loss:.4f}")
        save_and_show_images(generator, epoch)


train(train_dataset, EPOCHS)


🖥️ Device in use: ⚠️ No GPU found — running on CPU.
Found 8005 files.
✅ Epoch 0, Gen Loss: 1.0116, Disc Loss: 0.7100
✅ Epoch 1, Gen Loss: 2.3639, Disc Loss: 0.2183
✅ Epoch 2, Gen Loss: 3.5926, Disc Loss: 0.0801
✅ Epoch 3, Gen Loss: 3.9045, Disc Loss: 0.0410
✅ Epoch 4, Gen Loss: 4.8749, Disc Loss: 0.1373
